In [1]:
## IMPORTACIÓN GENERAL DE LIBRERIAS.
import re 
import sys
import json
import nltk
import geocoder
import requests
import warnings
import descartes

import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import datetime as DT
import geopandas as gpd
import matplotlib.pyplot as plt

# SKLEARN.
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# RANDOM FOREST.
from urllib.request import urlopen
from shapely.geometry import Point, Polygon
from sklearn.ensemble import RandomForestRegressor

# XGBOOST.
from xgboost import XGBClassifier
import xgboost as xgb

In [2]:
#NLTK: https://www.nltk.org
from langdetect import detect
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

#WORDCLOUD
from PIL import Image
from wordcloud import WordCloud

# CONFIGURACIÓN.
%matplotlib inline
plt.style.use('default')
pd.options.display.float_format = '{:20,.2f}'.format
warnings.filterwarnings('ignore')
sns.set(style="whitegrid") 
plt.rcParams['figure.figsize'] = (15, 10)
np.set_printoptions(threshold=sys.maxsize)

### =====================================================================
### RANDOM FOREST.
### =====================================================================

In [4]:
#OBTENEMOS LOS CSVs YA PROCESADOS EN LA PRIMER ETAPA.
original_train = pd.read_csv('data/processed/original_train.csv')
original_test = pd.read_csv('data/processed/original_test.csv')
original_sample_submission = pd.read_csv('data/sample_submission.csv')

In [5]:
original_train['is_train'] = np.random.uniform(0, 1, len(original_train)) <= .75
train, test = original_train[original_train['is_train']==True], original_train[original_train['is_train']==False]

print('Observamos para entrenar:', len(train))
print('OBservamos para probar:',len(test))

Observamos para entrenar: 5662
OBservamos para probar: 1951


In [6]:
#QUITAMOS LA COLUMNA AUXILIAR.
original_train.drop('is_train', axis = 1, inplace = True)
train.drop('is_train', axis = 1, inplace = True)
test.drop('is_train', axis = 1, inplace = True)

In [7]:
train_label = np.array(train['target'])
train_features = train.drop('target', axis = 1)

test_label = np.array(test['target'])
test_features = test.drop('target', axis = 1)

In [8]:
clf = RandomForestRegressor(n_jobs=2, random_state=0)

In [9]:
clf.fit(train_features, train_label)

RandomForestRegressor(n_jobs=2, random_state=0)

In [10]:
prediccion_train = clf.predict(test_features)

In [11]:
prediccion_train = np.where(prediccion_train>0.5, 1, 0)

In [12]:
#NO SIRVE, PENSAR OTRA FORMA PORQUE ESTO ES SOLO TRUE O FALSE!
# Calculamos el error absoluto.
errors = abs(prediccion_train - test_label)
# Calculamos el porcentaje de error.
mape = 100 * (errors / test_label)
# Calculate la precisión.
accuracy = 100 - np.mean(mape)
print('Precision:', round(accuracy, 2), '%.')

Precision: nan %.


In [13]:
prediccion_test = clf.predict(original_test)

In [14]:
prediccion_test = np.where(prediccion_test>0.5, 1, 0)

In [15]:
kaggle_submission = pd.DataFrame(original_test,columns = ['id'])
kaggle_submission["target"] = prediccion_test
kaggle_submission

,id,target
0,0,0
1,2,1
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [16]:
kaggle_submission.to_csv('data/submits/submission_RF_02.csv',index=False)

### =====================================================================
### FIN DEL RANDOM FOREST.
### =====================================================================

In [36]:
original_train = ''
original_test = ''
original_sample_submission = ''

### =====================================================================
### XGBOOST.
### =====================================================================

In [37]:
#OBTENEMOS LOS CSVs YA PROCESADOS EN LA PRIMER ETAPA.
original_train = pd.read_csv('data/processed/original_train.csv')
original_test = pd.read_csv('data/processed/original_test.csv')
original_sample_submission = pd.read_csv('data/sample_submission.csv')

In [38]:
# Resto de los labels.
Y = np.array(original_train['target'])
X = original_train.drop('target', axis = 1)

In [39]:
# MODELO ENSAMBLE 3.
trainX_M3, testX_M3, trainY_M3, testY_M3 = train_test_split(X, Y, test_size=0.2, random_state=50)
data_dmatrix_inst = xgb.DMatrix(data=trainX_M3,label=trainY_M3)
model3 = xgb.XGBRegressor(objective = 'reg:linear', 
                          n_estimators = 1500,
                          min_child_weight = 5,
                          learning_rate = 0.05017181127931773,
                          gamma = 9,
                          reg_lambda = 2,
                          max_depth = 6,
                          colsample_bytree = 0.7585033814547916, 
                          subsample_bytree = 0.9779760690574663)

In [40]:
model3.fit(trainX_M3,trainY_M3)
pred_M3 = model3.predict(testX_M3)

[20:12:35] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[20:12:35] WARNING: /workspace/src/learner.cc:480: 
Parameters: { subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[20:13:18] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


In [41]:
prediccion = model3.predict(original_test)

In [42]:
prediccion = np.where(prediccion>=0.5, 1, 0)

In [43]:
kaggle_submission = pd.DataFrame(original_test,columns = ['id'])
kaggle_submission["target"] = prediccion
kaggle_submission

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0


In [44]:
kaggle_submission.to_csv("data/submits/submission_XGB_03.csv", index=False)

### =====================================================================
### FIN DEL XGBOOST.
### =====================================================================